In [1]:
import requests
import regex as re
import time
import os.path

from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import urllib.request 
import json

# I believe this is no longer needed - was used for authentication a different way
#from lxml import html


In [ ]:
test_url = 'https://www.ravelry.com/patterns/library/love-note'
result = requests.get(test_url)
#if result.status_code == 200:
soup = BeautifulSoup(result.content, 'html.parser')

In [ ]:
photo_gallery = soup.find('div', class_ = 'photo_gallery_container').find_all('img')

In [ ]:
photo_gallery[0]

In [ ]:
photo_gallery[0]['src']

OK, code to get lists of photo gallery links for a given page is

In [ ]:
[image['src'] for image in soup.find('div', class_ = 'photo_gallery_container').find_all('img')]

"Projects" page for a given pattern is same as pattern link but with ```/people``` appended to it.

Now want to get several photos from the projects page. Since there are also in progress photos, let's look at the finished projects page. ```/people?view=cards&search=&view=cards&statuses%5B%5D=finished```

Unexpected problem here: some people only have photos of the balls/skeins of yarn they plan on using to make the sweater, not the finished item. Possible solution: get the photo neural net to identify balls of yarn and drop those photos?

Getting thumbnails for first projects page:

In [ ]:
people_url = 'https://www.ravelry.com/patterns/library/love-note/people?view=cards&search=&view=cards&statuses%5B%5D=finished'
result = requests.get(people_url)
people_soup = BeautifulSoup(result.content, 'html.parser')

In [ ]:
soup.find('div', class_ = 'photo_frame photo_gallery__frame real_photo').find('img')

In [ ]:
soup.find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')

In [ ]:
# Community image thumbnails
[card.find('img')['src'] for card in soup.find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]

8 user-submitted pictures (probably at least 4 of them the actual garment) will be enough for the test run.

Now to take a step back and scrape pattern links from the search page.

In [ ]:
search_url = 'https://www.ravelry.com/patterns/search#craft=crochet%7Cknitting&pc=sweater&sort=best&view=captioned_thumbs'
result = requests.get(search_url)
search_soup = BeautifulSoup(result.content, 'html.parser')

search_soup.find('div')

Search page requires login. Hmm. They have an API, which I'm going to request access to. In the meantime, fun with logins!

```<label for="user_login">Username</label>
<input autofocus="autofocus" id="user_login" name="user[login]" size="30" type="text" value="" style="background-image: url(&quot;data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR4nGP6zwAAAgcBApocMXEAAAAASUVORK5CYII=&quot;); cursor: auto;">```

```<label for="user_password">Password</label>
<input id="user_password" name="user[password]" size="30" type="password" style="background-image: url(&quot;data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR4nGP6zwAAAgcBApocMXEAAAAASUVORK5CYII=&quot;); cursor: auto;">```

```<meta content="BrjS4V87lBeD/pLeSy3xGn3mtGSMIQx2UKxE7bBeAV0=" id="authenticity-token" name="authenticity-token">```


In [ ]:

login = {'user_login': 'cheshious@gmail.com', 
        'user_password': 'angel1ne', 
        'authenticity_token': 'BrjS4V87lBeD/pLeSy3xGn3mtGSMIQx2UKxE7bBeAV0='}
session_requests = requests.session()
login_url = 'https://www.ravelry.com/account/login'
result = session_requests.get(login_url)

soup = BeautifulSoup(result.content, 'html.parser')
authenticity_token = soup.find('input')['value']
login['authenticity-token'] = authenticity_token

result = session_requests.post(login_url, 
                               data = login, 
                               headers = dict(referer=login_url))

In [ ]:
login['user[login]'] = 'cheshious@gmail.com'
login['user[password]'] = 'angel1ne'

In [ ]:
with session_requests as session:
    result = session.get(login_url)
    soup = BeautifulSoup(result.content, 'html.parser')
    authenticity_token = soup.find('input')['value']
    login['authenticity-token'] = authenticity_token
    result = session_requests.post(login_url, data = login)

In [ ]:
result.status_code

In [ ]:
soup = BeautifulSoup(result.content, 'html.parser')
soup

And they've got some anti-bot measures as well! Dang!

I'm going to cheat and right-click on some image links to get pattern pages I can scrape and play around with while I try to figure out their API.

In [ ]:
pattern_links = ['https://www.ravelry.com/patterns/library/stripes-32', 
                'https://www.ravelry.com/patterns/library/love-note', 
                'https://www.ravelry.com/patterns/library/soldotna-crop', 
                'https://www.ravelry.com/patterns/library/the-weekender-5', 
                'https://www.ravelry.com/patterns/library/ranunculus-3', 
                'https://www.ravelry.com/patterns/library/flax', 
                'https://www.ravelry.com/patterns/library/zweig', 
                'https://www.ravelry.com/patterns/library/pull-camille', 
                'https://www.ravelry.com/patterns/library/navelli', 
                'https://www.ravelry.com/patterns/library/soundtrack-2', 
                'https://www.ravelry.com/patterns/library/felix-pullover', 
                'https://www.ravelry.com/patterns/library/forestland', 
                'https://www.ravelry.com/patterns/library/super-simple-summer-sweater', 
                'https://www.ravelry.com/patterns/library/no-frills-sweater', 
                'https://www.ravelry.com/patterns/library/cumulus-blouse', 
                'https://www.ravelry.com/patterns/library/ursa-3', 
                'https://www.ravelry.com/patterns/library/fonda', 
                'https://www.ravelry.com/patterns/library/humulus-2', 
                'https://www.ravelry.com/patterns/library/mjuk-hettegenser-herre', 
                'https://www.ravelry.com/patterns/library/misurina', 
                'https://www.ravelry.com/patterns/library/city-limits-2', 
                'https://www.ravelry.com/patterns/library/yume', 
                'https://www.ravelry.com/patterns/library/so-faded', 
                'https://www.ravelry.com/patterns/library/the-throwover', 
                'https://www.ravelry.com/patterns/library/rift-7', 
                'https://www.ravelry.com/patterns/library/felix-cardigan', 
                'https://www.ravelry.com/patterns/library/shifty-2', 
                'https://www.ravelry.com/patterns/library/forager-4', 
                'https://www.ravelry.com/patterns/library/elton-3', 
                'https://www.ravelry.com/patterns/library/sunday-sweater-7', 
                'https://www.ravelry.com/patterns/library/the-throwback', 
                'https://www.ravelry.com/patterns/library/wool--honey', 
                'https://www.ravelry.com/patterns/library/comfort-fade-cardi', 
                'https://www.ravelry.com/patterns/library/magnolia-chunky-cardigan', 
                'https://www.ravelry.com/patterns/library/nurtured', 
                'https://www.ravelry.com/patterns/library/sipila', 
                'https://www.ravelry.com/patterns/library/sorrel-15', 
                'https://www.ravelry.com/patterns/library/trescao-jumper', 
                'https://www.ravelry.com/patterns/library/gardengate', 
                'https://www.ravelry.com/patterns/library/spark-5', 
                'https://www.ravelry.com/patterns/library/cozy-classic-raglan', 
                'https://www.ravelry.com/patterns/library/tecumseh-2', 
                'https://www.ravelry.com/patterns/library/the-weekender-light', 
                'https://www.ravelry.com/patterns/library/turtle-dove-3', 
                'https://www.ravelry.com/patterns/library/fog-whisperer-cardigan', 
                'https://www.ravelry.com/patterns/library/pink-velvet']

# First page of results at https://www.ravelry.com/patterns/search#craft=crochet%7Cknitting&availability=free%7Conline%7Cravelry&pc=sweater&photo=yes&sort=best&view=captioned_thumbs
# Knit or crochet, available online, sweater, has photos

In [ ]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}
search_stem = 'https://www.ravelry.com/patterns/search#craft=crochet%7Cknitting&availability=free%7Conline%7Cravelry&photo=yes&pc=sweater&sort=best&view=captioned_thumbs&page='
session_requests = requests.Session()
with session_requests as session:
    result = session.get(login_url, headers = user_agent)
    soup = BeautifulSoup(result.content, 'html.parser')
    authenticity_token = soup.find('input')['value']
    login['authenticity-token'] = authenticity_token
    session_requests.post(login_url, data = login)
    result = session.get(search_stem + '2', headers = user_agent)

In [ ]:
soup = BeautifulSoup(result.content, 'html.parser')

In [ ]:
soup

In [ ]:
# Getting pattern name from url:
pattern_scrapes = {}
for link in pattern_links:
    user_agent = {'User-agent': ua.random}
    name = re.search('(?<=library/)[-A-z0-9]+', link)
    result = requests.get(link, headers = user_agent)
    pattern_scrapes[name] = result
    time.sleep(30)

In [ ]:
print(len(pattern_scrapes), len(pattern_links))

In [ ]:
soup_dict = {}
for key in pattern_scrapes.keys():
    # key is currently a regex.Match object; getting string of what matched
    soup_dict[key.group()] = BeautifulSoup(pattern_scrapes[key].content, 'html.parser')

In [ ]:
pattern_dict = {}
for key in soup_dict.keys():
    pattern_dict[key] = {}
    pattern_dict[key]['gallery'] = [image['src'] for image in soup_dict[key].find('div', class_ = 'photo_gallery_container').find_all('img')]

In [ ]:
people_scrapes = {}
# Getting community photos of same project; first page only
for link in pattern_links:
    people_link = link + '/people?view=cards&search=&view=cards&statuses%5B%5D=finished'
    user_agent = {'User-agent': ua.random}
    name = re.search('(?<=library/)[-A-z0-9]+', link).group()
    result = requests.get(people_link, headers = user_agent)
    people_scrapes[name] = result
    time.sleep(30)

In [ ]:

# Getting second page of projects to test neural net
people_scrapes2 = {}
# Getting community photos of same project; first page only
for link in pattern_links:
    people_link = link + '/people?page=2&search=&statuses%5B%5D=finished&view=cards'
    user_agent = {'User-agent': ua.random}
    name = re.search('(?<=library/)[-A-z0-9]+', link).group()
    result = requests.get(people_link, headers = user_agent)
    people_scrapes2[name] = result
    time.sleep(30)

In [ ]:
# Turning raw results of community pages to BeautifulSoup objects
people_soup = {}
for key in people_scrapes.keys():
    people_soup[key] = BeautifulSoup(people_scrapes[key].content, 'html.parser')
people_soup2 = {}
for key in people_scrapes2.keys():
    people_soup2[key] = BeautifulSoup(people_scrapes2[key].content, 'html.parser')

In [ ]:
# Getting links from community pages; appending first to links list in dict and putting second in its own list
for key in pattern_dict.keys():
    pattern_dict[key]['gallery'] += [card.find('img')['src'] for card in people_soup[key].find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]
    pattern_dict[key]['test links'] = [card.find('img')['src'] for card in people_soup2[key].find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]
    

In [ ]:
test_pic = urllib.request.urlretrieve('https://images4-f.ravelrycache.com/uploads/dreareneeknits/743362891/Custom-8_thumbnail.jpg', 'test.jpg')


In [ ]:
with open('text.txt', 'w') as fp:
    fp.write('I am a test')

In [ ]:
os.chdir('proj05')

In [ ]:
os.getcwd()

In [ ]:
for pattern in pattern_dict.keys():
    train_count = 1
    test_count = 100
    for link in pattern_dict[pattern]['gallery']:
        file_name = pattern + '_' + str(train_count).zfill(3) + '.jpg'
        file_path = os.path.join('img/', file_name)
        try:
            pic = urllib.request.urlretrieve(link, file_path)
        except:
            pass
        train_count += 1
        time.sleep(15)
    for link in pattern_dict[pattern]['test links']:
        file_name = pattern + '_' + str(test_count).zfill(3) + '.jpg'
        file_path = os.path.join('img/test/', file_name)
        try:
            pic = urllib.request.urlretrieve(link, file_name)
        except:
            pass
        test_count += 1
        time.sleep(15)

Something is up; images are duplicated in folder AND are in '/test/'.

In [ ]:
# Comparing links:
gallery = [image['src'] for image in soup_dict['stripes-32'].find('div', class_ = 'photo_gallery_container').find_all('img')]
projects1 = [card.find('img')['src'] for card in people_soup['stripes-32'].find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]
projects2 = [card.find('img')['src'] for card in people_soup2['stripes-32'].find_all('div', class_ = 'photo_frame photo_gallery__frame real_photo')]
gallery

Really not clear why results are identical but reordered in various sets/various pages. Going to work on retrieving things from API and then return to this if/when I need to scrape directly.

## Hybrid method
API is great for getting patterns including permalink and link to first image as small2. (Requires OAuth to get projects by pattern, which is a major pain.) 

In [2]:
pattern_dict = {'womens-knitted-sweater-2': {'id': 1088978,
  'url': 'https://www.novitaknits.com/fi',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/knitabitknitwit/744566300/upload_small2']},
 'hooded-baby-cardigan-5': {'id': 1089008,
  'url': 'https://www.novitaknits.com/fi',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/knitabitknitwit/744588460/upload_small2']},
 'love-note': {'id': 927223,
  'url': 'http://www.ravelry.com/stores/tin-can-knits-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/tincanknits/624440924/TCK-lovenote-01a_small2.jpg']},
 'womens-cabled-cardigan': {'id': 1088911,
  'url': 'https://www.novitaknits.com/fi',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/knitabitknitwit/744521115/upload_small2']},
 'soldotna-crop': {'id': 910492,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/612664255/Attachment-1_2_small2.jpeg']},
 'the-weekender-5': {'id': 788421,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/481027443/IMG_0617_small2.JPG']},
 'ranunculus-3': {'id': 754478,
  'url': 'http://www.ravelry.com/stores/midori-hirose-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/MidoriHirose/475388370/Blob_small2']},
 'flax': {'id': 443533,
  'url': 'http://tincanknits.com/thesimplecollection.html',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/tincanknits/731184306/FlaxSweater-SC-TCK-3536_small2.jpg']},
 'flax-light': {'id': 632425,
  'url': 'http://www.tincanknits.com/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/tincanknits/338183095/SC-flaxlight-07a_small2.jpg']},
 'zweig': {'id': 778932,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/Knitosophy/474367583/DSC02703_small2.JPG']},
 'cabin-life-cardigan': {'id': 1088754,
  'url': '',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jojilocat/744326942/WN7A7886_small2.jpg']},
 'navelli': {'id': 927665,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/624513479/1B8DAB2E-4617-460F-AB59-D76E5270B818_small2.jpg']},
 'soundtrack-2': {'id': 1037794,
  'url': 'http://www.ravelry.com/stores/marie-greene-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/mariegreene/706765307/Soundtrack_Cover_small2.jpg']},
 'felix-pullover': {'id': 894972,
  'url': 'http://www.ravelry.com/stores/savory-knitting',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/savoryknitting/599113250/IMG_8037_small2.jpg']},
 'super-simple-summer-sweater': {'id': 944225,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jojilocat/636063575/WN7A6565_small2.jpg']},
 'no-frills-sweater': {'id': 710742,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/685972518/hvid9_small2.jpg']},
 'cumulus-blouse': {'id': 840489,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/649817159/rd3_small2.jpg']},
 'ursa-3': {'id': 916577,
  'url': 'http://www.ravelry.com/stores/jacqueline-cieslak-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/JacquelineCieslak/616264160/ursa-ravelry_small2.jpg']},
 'humulus-2': {'id': 790875,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/476854643/22429752_1810582145900620_917657975114432512_n_small2.jpg']},
 'aimee-cardigan': {'id': 1088763,
  'url': '',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jojilocat/744357776/WN7A7156_small2.jpg']},
 'misurina': {'id': 1056994,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/721285822/F6070E3A-B7EC-4126-BF70-712DAE7D3AFF-4161AA11-9C5A-407E-9A18-2D77FCB9EBAB_small2.jpg']},
 'tornado-2': {'id': 1087901,
  'url': 'http://www.ravelry.com/stores/astrid-muller-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/rubator/743717113/IMG_1081_small2.jpg']},
 'city-limits-2': {'id': 1016281,
  'url': 'http://www.ravelry.com/stores/tanis-fiber-arts-2',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/tanisfiberarts/686318527/fullsizeoutput_40da_small2.jpeg']},
 'yume': {'id': 839122,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/526545028/IMG_1552-001_small2.JPG']},
 'so-faded': {'id': 742300,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/442849891/untitled-78_small2.jpg']},
 'the-throwover': {'id': 1014235,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/688641604/6_small2.jpg']},
 'felix-cardigan': {'id': 957863,
  'url': 'http://www.ravelry.com/stores/savory-knitting',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/savoryknitting/645646327/IMG_9495_small2.jpg']},
 'rift-7': {'id': 944281,
  'url': 'http://www.ravelry.com/stores/jacqueline-cieslak-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/JacquelineCieslak/635464340/IMG_2130_copy_small2.jpg']},
 'shifty-2': {'id': 897128,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/600733310/IMG_1227_small2.JPG']},
 'forestland': {'id': 1086589,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/742788273/fullsizeoutput_8b4_small2.jpeg']},
 'sunday-sweater-7': {'id': 888409,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/592040660/Sunday_Sweater_1_small2.jpg']},
 'elton-3': {'id': 861747,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jojilocat/546299631/WN7A4002_small2.jpg']},
 'wool--honey': {'id': 838914,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/525847812/untitled-48_small2.jpg']},
 'the-throwback': {'id': 857539,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/543345334/IMG_1058_small2.JPG']},
 'magnolia-chunky-cardigan': {'id': 959956,
  'url': 'http://www.ravelry.com/stores/camilla-vad-2',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/CamillaVad/707378611/B37103B6-BCAE-4C1B-93E4-897BDE86D899_small2.jpeg']},
 'comfort-fade-cardi': {'id': 781137,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/475398387/1-148_small2.jpg']},
 'nurtured': {'id': 865227,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/544579894/untitled-41_small2.jpg']},
 'changing-paths': {'id': 1088759,
  'url': '',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jojilocat/744344693/WN7A7602_small2.jpg']},
 'sipila': {'id': 833778,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/521150793/Screen_Shot_2018-05-08_at_6.23.25_PM_small2.png']},
 'evergreen-cardigan-2': {'id': 1088749,
  'url': '',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/veera/743911302/IMG_6711-2-Editar_small2.jpg']},
 'modern-vintage-sweater': {'id': 1088758,
  'url': '',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/veera/744184026/IMG_6496_small2.jpg']},
 'sorrel-15': {'id': 984587,
  'url': '',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/WoolandPine/667044228/899BB01A-E44A-446A-B66D-9C3ACA230D99_small2.jpeg']},
 'trescao-jumper': {'id': 989857,
  'url': 'https://www.ravelry.com/stores/along-avec-anna---knitting-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/alongavecanna/669016467/1F8344CC-6029-467A-98AB-3A41ACDB1F2C_small2.jpeg']},
 'forager-4': {'id': 1085622,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/742107957/IMG_8663_small2.jpg']},
 'spark-5': {'id': 1050845,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/716587487/IMG_1848_small2.JPG']},
 'gardengate': {'id': 939945,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/633312211/fullsizeoutput_10b8_small2.jpeg']},
 'cozy-classic-raglan': {'id': 988974,
  'url': 'https://www.ravelry.com/stores/jessie-mae-martinson-jessie-maed-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jessiemaed/670299849/68E71C78-2DED-49C4-8710-BE5540CAAE81_small2.jpg']},
 'tecumseh-2': {'id': 822497,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/511690541/Facetune_16-03-2018-14-32-22_small2.JPG']},
 'the-weekender-light': {'id': 1043109,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/710789924/IMG_1817_small2.JPG']},
 'turtle-dove-3': {'id': 855998,
  'url': 'https://www.ravelry.com/stores/espace-tricot-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/EspaceTricot/542040980/turtledove_small2.jpg']},
 'strange-brew': {'id': 775740,
  'url': 'http://www.ravelry.com/stores/tin-can-knits-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/tincanknits/549013092/SB-strangebrew-09a_small2.jpg']},
 'pink-velvet': {'id': 1006482,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/682570494/pinkvelvet-27_copy_small2.jpg']},
 'carbeth-cardigan': {'id': 814316,
  'url': 'http://www.ravelry.com/stores/kate-davies-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/KateDaviesDesigns/505014435/Blob_small2']},
 'mariechen': {'id': 918059,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/617131502/IMG_1116_small2.JPG']},
 'fern--feather': {'id': 775391,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/hannahbelle/471151248/5C0A4269_small2.jpg']},
 'goldwing-2': {'id': 1012685,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/735917239/F4C0A557-160C-4948-BA3D-A1499FD0713D_small2.jpeg']},
 'marettimo': {'id': 848725,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/535822594/E7B35A63-4CBD-4B08-B854-078D06FB117E_small2.JPG']},
 'jupiter-crop': {'id': 1033986,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/703258812/photo4_small2.jpg']},
 'sunday-cardigan-4': {'id': 904870,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/607249155/Sunday_Cardigan_1_small2.jpg']},
 'carbeth': {'id': 801053,
  'url': 'http://www.ravelry.com/stores/kate-davies-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/KateDaviesDesigns/493021393/carbeth_01_copy_small2.jpg']},
 'arboreal': {'id': 783068,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/Lovewool-Knits/476869819/DSC_0071__2__small2.JPG']},
 'bean--olive': {'id': 1072478,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/732783535/IMG_0184_small2.jpeg']},
 'stripes-32': {'id': 1087512,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/743362891/Custom-8_small2.jpg']},
 'ghost-horses': {'id': 960660,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/643636858/6005349B-DDCC-4C40-BF1B-5D2288407537_small2.JPG']},
 'koivua': {'id': 906148,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/608179569/IMG_7513_small2.jpg']},
 'magnolia-26': {'id': 850976,
  'url': 'http://www.ravelry.com/stores/camilla-vad-2',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/CamillaVad/639154796/E826A140-2DA9-44FE-AB6C-5F59D44F0AE0_small2.jpeg']},
 'seven-sisters-top': {'id': 1013695,
  'url': 'http://blueskyfibers.com/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/BlueSkyFibers/685421860/P_SevenSistersTop_R_small2.jpg']},
 'nordiska': {'id': 887688,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/591935633/JPEG_image-E20B5F757110-4_small2.jpeg']},
 'doocot': {'id': 872372,
  'url': 'http://www.ravelry.com/stores/kate-davies-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/KDDandco/728364385/Doocot_two-99_small2.jpg']},
 'big-love-5': {'id': 889407,
  'url': 'http://www.ravelry.com/stores/fall-masche-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/fallmasche/592252642/IMG_9894_2_small2.jpeg']},
 'missoni-accomplished': {'id': 927710,
  'url': 'https://www.ravelry.com/stores/espace-tricot-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/EspaceTricot/624694128/L1000835_small2.jpg']},
 'stonecrop-pullover': {'id': 952303,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/641379056/Test-45_copy_small2.jpg']},
 'french-macaroon-2': {'id': 771194,
  'url': 'http://www.ravelry.com/stores/the-noble-thread',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/TheNobleThread/467869434/macaron2_small2.jpg']},
 'simple-hug-cardi': {'id': 1040638,
  'url': 'http://www.ravelry.com/stores/cozy-up-designs-by-the-stitchinsisters-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/cozyupknits/716890363/Simplehug2_small2.jpg']},
 'lady-fingers-sweater': {'id': 1024795,
  'url': 'http://www.ravelry.com/stores/morgan-woltersdorf',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/Morthunder/696780095/IMG_7326_small2.jpg']},
 'spector-2': {'id': 884555,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jojilocat/588957772/WN7A4087_small2.jpg']},
 'aldous-2': {'id': 961755,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/647718192/IMG_0652_small2.JPG']},
 'sunbeam-sweater-2': {'id': 1088762,
  'url': '',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/veera/744191159/IMG_7123_small2.jpg']},
 'harvest-10': {'id': 456074,
  'url': 'http://www.ravelry.com/stores/tin-can-knits-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/tincanknits/200601957/SC-harvest-14_small2.jpg']},
 'norwegian-fir-top-down-cardigan': {'id': 705322,
  'url': 'http://www.ravelry.com/stores/oge-knitwear-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/oge-designs/409437768/024_small2.JPG']},
 'novice-sweater': {'id': 873567,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/723441535/jensen1_small2.jpg']},
 'like-a-cloud': {'id': 836806,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jojilocat/524175746/WN7A2621-2_small2.jpg']},
 'graphic-elements': {'id': 1087761,
  'url': 'https://www.ravelry.com/stores/narrow-path-knitting-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/TamyGore/743013096/20201112_212819_small2.jpg']},
 'le-pouf-2': {'id': 965339,
  'url': 'http://shop.hedgehogfibres.com/',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/HedgehogFibres/650516209/mb_hedgehog_fibres_2019-09-21_012_small2.JPG']},
 'newleaf': {'id': 924996,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/622914209/fullsizeoutput_b27_small2.jpeg']},
 'diaphanous-raglan': {'id': 944366,
  'url': 'https://www.ravelry.com/stores/jessie-mae-martinson-jessie-maed-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jessiemaed/626040459/ACS_0601_small2.JPG']},
 'vellichor': {'id': 949070,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/639596426/untitled-34_copy_small2.jpg']},
 'metropolis-3': {'id': 966600,
  'url': 'http://www.ravelry.com/stores/tanis-fiber-arts-2',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/tanisfiberarts/651389508/cover_small2.jpg']},
 'purl-strings-2': {'id': 1040101,
  'url': 'http://www.ravelry.com/stores/heidi-kirrmaier-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/HeidiKdesigns/708140761/3070B6CD-18A4-4579-AA7D-68B9A740B1FF_small2.jpeg']},
 'hinterland-5': {'id': 991497,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/670304525/fullsizeoutput_1457_small2.jpeg']},
 'deschain': {'id': 732801,
  'url': 'https://www.ravelry.com/stores/quince--co',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/quinceandco/433266829/quince-co-deschain-leila-raabe-knitting-pattern-kestrel-1_small2.jpg']},
 'foxtrot-cardigan': {'id': 930644,
  'url': 'http://www.ravelry.com/stores/marie-greene-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/mariegreene/626978245/Foxtrot1_small2.jpg']},
 'sunset-highway': {'id': 760196,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/458308817/SHS4_small2.jpg']},
 'vanilla-sweater-2': {'id': 1032801,
  'url': 'https://www.ravelry.com/stores/the-woolly-thistle',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/nhkclaire/702729320/vanilla_sweater_gold_group-removebg-preview_small2.png']},
 'poet': {'id': 863944,
  'url': 'http://www.ravelry.com/stores/lunaknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lainemagazine/541658512/laine_6_sarinordlund_2_jh_small2.jpg']},
 'ooh-la-la-4': {'id': 1061270,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/718150510/IMG_4852_small2.jpg']},
 'a-warm-welcome': {'id': 1084910,
  'url': 'http://www.ravelry.com/stores/von-hinterm-stein-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/HintermStein/741706305/000_welcome-baby-by-hinterm-stein_1ad_small2.jpg']},
 'mycroft': {'id': 913969,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/613896897/fullsizeoutput_29ef_small2.jpeg']},
 'birkin-3': {'id': 747810,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lainemagazine/444790659/yoked_1_jh_small2.jpg']},
 'junction-3': {'id': 1066400,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/728230807/IMG_1935_small2.JPG']},
 'dont-ask': {'id': 1036695,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/696230344/IMG_2066_small2.jpg']},
 'vintersol': {'id': 800080,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/492119599/A8B0586C-0DE7-4666-BB17-20B0335D3504_small2.jpeg']},
 'clapton': {'id': 1065145,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/727326941/IMG_7544_small2.jpg']},
 'retrograde-4': {'id': 1087759,
  'url': 'https://www.ravelry.com/stores/narrow-path-knitting-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/TamyGore/742999975/20201111_173239_small2.jpg']},
 'ninilchik-swoncho': {'id': 726789,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/429841042/E4011B74-6B08-46A2-A47E-6045E3ACD555_small2.jpg']},
 'meadow-moon': {'id': 1037791,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/706040317/DSC_0902_small2.jpeg']},
 'darkwater': {'id': 891054,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/629595180/fullsizeoutput_e99_small2.jpeg']},
 'cozy-classic-light': {'id': 1066011,
  'url': 'https://www.ravelry.com/stores/jessie-mae-martinson-jessie-maed-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jessiemaed/728065904/0504C028-FA24-48E2-A1F6-316EBEF7A786_small2.jpeg']},
 'newborn-vertebrae': {'id': 270527,
  'url': 'https://www.ravelry.com/stores/kelly-van-niekerks-ravelry-store',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/MVick/239467058/image_small2.jpg']},
 'hitofude-cardigan': {'id': 440784,
  'url': 'http://www.ravelry.com/stores/please-dont-eat-the-daisies',
  'photo urls': ['https://images4-g.ravelrycache.com/flickr/9/6/7/9679091926/9679091926_n.jpg']},
 'turtle-dove-ii': {'id': 966254,
  'url': 'https://www.ravelry.com/stores/espace-tricot-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/EspaceTricot/651227677/157A3DA1-82D6-4E8C-800A-1BC59AD91814_small2.jpeg']},
 'no-frills-cardigan': {'id': 772466,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/468972235/Ingen_Dikkedarer_Cardigan_1_small2.jpg']},
 'pengweeno': {'id': 941487,
  'url': 'http://www.ravelry.com/stores/westknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/westknits/634325221/IMG_1555_small2.jpg']},
 'cedar-pullover': {'id': 970781,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jojilocat/654208892/WN7A7958-2-4_small2.jpg']},
 'mini-unbearable-hoodie': {'id': 1064541,
  'url': None,
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/bonjourlesgarcons/724949312/Mini_Unbearable_03_small2.jpeg']},
 'ginny': {'id': 987283,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/665989795/untitled-74_small2.jpg']},
 'ingalls': {'id': 859590,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/544851861/E4F59D81-EA62-40A6-B28C-FA44AE371AB0_small2.JPG']},
 'puerperium-cardigan': {'id': 243083,
  'url': 'https://www.ravelry.com/stores/kelly-van-niekerks-ravelry-store',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/gurnard/61908451/IMG_1200_small2.JPG']},
 'riddari': {'id': 106061,
  'url': 'https://www.ravelry.com/stores/istex-lopi',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/annamargret5/392366718/L28_03_Riddari_1_small2.jpg']},
 'alden-2': {'id': 1080758,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/738806912/fullsizeoutput_3a1_small2.jpeg']},
 'september-sweater-2': {'id': 950920,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/640980610/sep8_small2.jpg']},
 'ankers-sweater': {'id': 731318,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/652923317/jens3_small2.JPG']},
 'rug-11': {'id': 785512,
  'url': 'http://www.ravelry.com/stores/junko-okamoto-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/JunkoOkamoto/477555826/4AB30E54-D269-4B6F-B317-5331653CECFF_small2.jpeg']},
 'susurrus-3': {'id': 1002224,
  'url': '',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/sophrscott/678389383/SUSURRUS_JOJI_LOCATELLI_PPQ32_SPRING2020_01_small2.jpg']},
 'beekeeper-cardigan': {'id': 835285,
  'url': 'http://www.ravelry.com/stores/marie-greene-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/mariegreene/527007706/P3390082_small2.JPG']},
 'sonrae': {'id': 995516,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/675462214/fullsizeoutput_17fa_small2.jpeg']},
 'the-v-back-tee': {'id': 855203,
  'url': 'https://www.ravelry.com/stores/knitosophy-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Knitosophy/540958218/Vback_today.2_small2.jpg']},
 'little-love-3': {'id': 951728,
  'url': 'http://www.ravelry.com/stores/fall-masche-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/fallmasche/641478846/IMG_2516_small2.jpeg']},
 'nigella-8': {'id': 1089059,
  'url': 'https://www.ravelry.com/stores/artherapy',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Tibisay/721336047/00AB8838-5F1D-443A-91B8-F231B6011B03-DFF706FE-5355-4203-BB6C-7CAD37E2B724_small2.jpeg']},
 'winterfell-cardigan': {'id': 819667,
  'url': 'https://www.ravelry.com/stores/katrin-schneider-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/Ahribokatrinchen/509781444/Foto_08.03.18__18_17_09_small2.jpg']},
 'copenhagen-cardigan': {'id': 921229,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/619734726/cop2_small2.jpg']},
 'november20': {'id': 1086012,
  'url': 'http://www.ravelry.com/stores/fall-masche-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/fallmasche/742354214/3025B289-A16C-4C94-93E2-0F78423F74E4-548F9DF0-871F-44FD-AC21-016FD28D9C6D_small2.jpeg']},
 'so-faded-pint-sized': {'id': 742301,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/442845358/SFPint_small2.jpg']},
 'whitehorse-2': {'id': 772417,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/468902645/whitehorse_2_small2.jpg']},
 'glenbarrow': {'id': 1058751,
  'url': 'http://www.ravelry.com/stores/stolen-stitches',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Littlefellers/722336807/IMG_9067_small2.jpeg']},
 'mountain-mist-2': {'id': 865522,
  'url': 'http://tincanknits.com/collection-strangebrew.html',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/tincanknits/546518161/SB-mountainmist-b-5_small2.jpg']},
 'fonda': {'id': 1087116,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/743205196/FE968652-D960-4100-8EFA-FB5A162ABE42-06201D94-7A60-4C70-B32F-52C4DA8F93C4_small2.jpg']},
 'manou': {'id': 960835,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/647561650/IMG_0776_small2.JPG']},
 'sunshine-coast': {'id': 683027,
  'url': 'http://www.ravelry.com/stores/heidi-kirrmaier-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/flickr/2/8/4/28462957156/28462957156_n.jpg']},
 'night-groove': {'id': 1005915,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/677744384/IMG_6978_small2.jpg']},
 'colour-block-patchwork-cardigan': {'id': 1048092,
  'url': 'https://www.jwanderson.com/gb/cardigan-pattern',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/ellagordondesigns/714607489/FEC33676-DD41-4C95-B16B-E42A2629608C_small2.jpeg']},
 'column-3': {'id': 901644,
  'url': 'https://www.ravelry.com/stores/plain-needle-works-2',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/pucco/604879836/IMG_1828_small2.jpg']},
 'ravello': {'id': 427616,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/179998540/IMG_3657-001_small2.JPG']},
 'campside-cardi': {'id': 594331,
  'url': 'http://www.ravelry.com/stores/twolittleplums',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/AliciaPlum/314968912/IMG_8540_small2.jpg']},
 'solvi': {'id': 1075929,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/735321731/fullsizeoutput_16f_small2.jpeg']},
 'my-boy-lollipop': {'id': 869199,
  'url': 'http://gettingpurlywithit.com/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/GettingPurly/551656925/MyBoyLollipopPink_-_1_small2.jpg']},
 'main-squeeze-cardigan': {'id': 958486,
  'url': 'https://www.moderndailyknitting.com/shop/mason-dixon-knitting-field-guide-no-12-big-joy/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/mdkjanet19/644696062/MDK_FG12_07_Cardigan1_4111_650x650_small2.jpg']},
 'sawyer-7': {'id': 1086312,
  'url': 'http://www.ravelry.com/stores/lunaknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/sarijaotto/742647769/upload_small2']},
 'ankers-sweater---my-size': {'id': 738121,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/667168824/anker2_small2.jpg']},
 'paloma-15': {'id': 1000531,
  'url': 'https://www.ravelry.com/stores/espace-tricot-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/EspaceTricot/678682293/ABF7AEFE-7BD6-4BEE-A349-0D04796B4329_small2.jpeg']},
 'boxy': {'id': 337715,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jojilocat/114984679/Boxy_01_small2.jpg']},
 'chauncey-2': {'id': 891408,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/595875454/IMG_8171-001_small2.JPG']},
 'knitorious-rbg': {'id': 885770,
  'url': 'https://www.ravelry.com/stores/parkknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/parknknitofficial/589747306/IMG_3686_2_small2.jpg']},
 'louisiana-sweater': {'id': 993179,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/744655071/louisiana1_2_small2.jpg']},
 'gramps': {'id': 259848,
  'url': 'http://www.tincanknits.com/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/tincanknits/243461219/9M-gramps-27_small2.jpg']},
 'brightfeather': {'id': 986748,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/665676886/fullsizeoutput_222_small2.jpeg']},
 'sommarkoftan-2020--the-summer-cardigan-2020': {'id': 1045126,
  'url': 'http://www.jarbo.se/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/MajasManufaktur/712135443/1678C43E-22DF-431E-84F9-7CCEC2967620_small2.jpeg']},
 'silverlining': {'id': 994876,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/674174618/fullsizeoutput_17ce_small2.jpeg']},
 'panglossian-sweater': {'id': 1035803,
  'url': 'http://www.ravelry.com/stores/morgan-woltersdorf',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Morthunder/705161646/99687D29-2247-496B-AC1F-57013047217B_2_small2.JPG']},
 'telja': {'id': 723693,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Lovewool-Knits/426722913/IMG_0180_small2.jpg']},
 'tiberius': {'id': 1076945,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/735994970/IMG_9929_small2.jpg']},
 'midcoast': {'id': 973002,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/655683804/fullsizeoutput_1271_small2.jpeg']},
 'celosias': {'id': 1044943,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/712276874/DSC_0366_small2.jpeg']},
 'autumn-league-pullover': {'id': 852742,
  'url': 'https://www.ravelry.com/stores/two-of-wands',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/TwoOfWands/539344556/AF72F836-51DF-4F54-9E81-A0FE5731D3E6_small2.JPG']},
 'understated-2': {'id': 912355,
  'url': 'https://www.interpretationsbyjojiandveera.com',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jojilocat/612857756/BL3A8334_small2.jpg']},
 'sunday-cardigan---mohair-edition': {'id': 928976,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/625784204/sun5_small2.jpg']},
 'letho': {'id': 1062836,
  'url': None,
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Moonstruckmermaid/725041836/Shawl_Collar_Cardi_back_2_small2.jpg']},
 'snow-falling-on-cedars': {'id': 1088958,
  'url': None,
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/TuffCityKnits/744553239/4B4388D1-5F15-49B7-9B52-376C76DD40E9_small2.jpg']},
 'strathcona-sweater': {'id': 781496,
  'url': 'http://www.ravelry.com/stores/good-night-day',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/goodnightdayknits/508401160/AWxGND20_small2.jpg']},
 'whitmoor-sweater': {'id': 1004612,
  'url': None,
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/tailoressstudio/681424934/whitmoor_cover_small2.jpg']},
 'treysta': {'id': 777540,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Lovewool-Knits/472751977/DSC_1260_small2.JPG']},
 'rose-74': {'id': 819713,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/508412234/untitled-6_small2.jpg']},
 'party-top': {'id': 861267,
  'url': 'https://www.ravelry.com/stores/abbyeknits-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/abbyedahl/545870252/Screen_Shot_2018-07-25_at_5.06.03_PM_small2.png']},
 'altheda-2': {'id': 932512,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lovewool-knits/656849719/fullsizeoutput_1291_small2.jpeg']},
 'vega-8': {'id': 1085638,
  'url': None,
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/tornedalsfrun/742156115/03A21263-C3A7-4497-BFE4-D4AD78108708_small2.jpeg']},
 'floozy-2': {'id': 851767,
  'url': 'http://www.ravelry.com/stores/truly-myrtle-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/trulymyrtle/538160111/IMG_5439_small2.jpg']},
 'rosa-cardigan-5': {'id': 927582,
  'url': 'https://www.ravelry.com/stores/along-avec-anna---knitting-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/alongavecanna/622486147/IMG_7070_small2.jpg']},
 'simple-something': {'id': 1000790,
  'url': 'http://www.ravelry.com/stores/fall-masche-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/fallmasche/678198974/FEA437EF-C9CE-460C-A67F-5A715A90BA32_small2.jpeg']},
 'niamh-11': {'id': 932468,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lilalu/628256287/IMG_7926_small2.JPG']},
 'aurealis-2': {'id': 1022518,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/lovewool-knits/694894841/DSC_0762_small2.jpeg']},
 'april-cardigan-2': {'id': 1025270,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/697258781/april1_small2.jpg']},
 'vintage-83': {'id': 1023353,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/dreareneeknits/695454711/4_small2.jpg']},
 'cambito': {'id': 1062954,
  'url': 'https://www.ravelry.com/stores/katrin-schneider-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/Ahribokatrinchen/725660791/Foto_28.08.20__15_49_27_small2.jpg']},
 'nuuk-3': {'id': 714391,
  'url': 'https://lainemagazine.com/',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lainemagazine/514633159/nuuk_1_small2.jpg']},
 'ursina': {'id': 1074520,
  'url': 'http://www.ravelry.com/stores/jacqueline-cieslak-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/JacquelineCieslak/733331058/cover_small2.jpg']},
 'the-classic-2': {'id': 1072713,
  'url': 'https://www.ravelry.com/stores/espace-tricot-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/EspaceTricot/732898113/IMG_0019_small2.jpeg']},
 'dog-star': {'id': 777830,
  'url': 'http://www.tincanknits.com/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/tincanknits/471451601/SB-dogstar-01b_small2.jpg']},
 'very-v-neck-raglan': {'id': 1048702,
  'url': 'https://www.ravelry.com/stores/jessie-mae-martinson-jessie-maed-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jessiemaed/715377113/76204722-C071-48B0-AEF7-0E8D84762A5B_small2.jpeg']},
 'teddy-bear-sweater-4': {'id': 710751,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/681145434/bamse6_small2.jpg']},
 'the-easy-bulky-one': {'id': 808102,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jojilocat/499874822/WN7A1310_small2.jpg']},
 'gloam': {'id': 1041807,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/boylandknitworks/709669672/photo1_8_small2.jpg']},
 'puntal': {'id': 1076435,
  'url': None,
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/tejereningles/735699391/Puntal_main_square_800px__1__small2.jpg']},
 'ankers-jacket': {'id': 736343,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/661225807/anker1_small2.jpg']},
 'homebody': {'id': 1019395,
  'url': 'http://www.ravelry.com/stores/heidi-kirrmaier-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/HeidiKdesigns/689980882/fullsizeoutput_aae4_small2.jpeg']},
 'guell': {'id': 1084113,
  'url': 'http://www.ravelry.com/stores/pelykh-natalie-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/knitforsweet/741162501/36_small2.jpg']},
 'stockholm-sweater-2': {'id': 852155,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/697283965/StockholmCropped_small2.jpg']},
 'lille-dahlia-solo': {'id': 918137,
  'url': 'https://www.ravelry.com/stores/lene-holme-samse-at-ravelry',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/holme/617373865/Lille_Dahlia_0_small2.jpg']},
 'dreyma': {'id': 814164,
  'url': 'http://www.ravelry.com/stores/knitlovewool',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/GriffinCreek/504886123/DSCF8715_edit_small2.jpg']},
 'sorbet-cardigan': {'id': 862327,
  'url': 'https://millefrydknitwear.dk/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Rabarberkage/613434827/EFF5AEC6-93FA-4801-AB68-FD64453AD47D_small2.jpeg']},
 'guthrie': {'id': 840171,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/527619578/664C25DA-4769-4B41-9F1E-723B2881321F_small2.JPG']},
 'lys-7': {'id': 914367,
  'url': 'https://www.ravelry.com/stores/dreareneeknits',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/614294002/IMG_1298_small2.JPG']},
 'great-love': {'id': 900563,
  'url': 'http://www.ravelry.com/stores/fall-masche-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/fallmasche/601408534/IMG_0264_small2.jpeg']},
 'rosa-little': {'id': 951539,
  'url': 'https://www.ravelry.com/stores/along-avec-anna---knitting-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/alongavecanna/641209146/_D725011_small2.JPG']},
 'suolaulu-sweater': {'id': 1061645,
  'url': None,
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/Ihtiriekkoknits/723419941/529DB8F2-5629-4758-933E-329AA2377204_small2.jpeg']},
 'beyond-the-dunes': {'id': 950592,
  'url': 'http://www.ravelry.com/stores/von-hinterm-stein-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/HintermStein/640649624/00_summerdunes-by-hinterm-stein-5a_small2.jpg']},
 'jaycee': {'id': 824626,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/514125321/IMG_1253-001_small2.JPG']},
 'mabrey-sweater': {'id': 1089027,
  'url': 'https://www.ravelry.com/stores/september-knits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/rachaelmreese/744606572/October2020pullover-4_small2.jpg']},
 'tsubaki-pullover': {'id': 993793,
  'url': 'http://www.ravelry.com/stores/please-dont-eat-the-daisies',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/roko/669764198/_2_small2.jpeg']},
 'glenmore': {'id': 1021417,
  'url': 'http://www.ravelry.com/stores/ysolda-teague-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/bleideag/694061153/Glenmore_indoor_Georsan__5_of_8__small2.jpg']},
 'alaska-sweater': {'id': 904228,
  'url': 'https://www.ravelry.com/stores/camille-descoteaux-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/camideco/606681293/camilledescoteauxALASKA_3_small2.jpg']},
 'all-together-sweater': {'id': 1039799,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jojilocat/708287472/WN7A4219_small2.jpg']},
 'reagan': {'id': 599723,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/318759062/IMG_7102_small2.JPG']},
 'girlfriends-cardigan-anke': {'id': 798509,
  'url': 'http://www.ravelry.com/stores/fall-masche-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/fallmasche/489618115/Blob_small2']},
 'easy-eyelet-yoke-sweater': {'id': 914468,
  'url': 'https://www.ravelry.com/stores/knitatude---chantal-miyagishima-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/Knitatude87/614559631/ACS_0011_small2.JPG']},
 'gracious-4': {'id': 948359,
  'url': 'https://www.ravelry.com/stores/espace-tricot-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/EspaceTricot/639291019/Untitled-8_small2.jpg']},
 'elorie': {'id': 969683,
  'url': 'http://www.ravelry.com/stores/blue-bee-studio',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/bluebeestudio/653268580/robert_1_small2.png']},
 'le-pouf-sweater': {'id': 977413,
  'url': 'http://shop.hedgehogfibres.com/',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/HedgehogFibres/658920661/mb_hedgehog_2019-11-01_027_small2.JPG']},
 'the-purl-code': {'id': 919964,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/617130418/IMG_0875_small2.JPG']},
 'high-lonesome': {'id': 1067494,
  'url': 'https://www.ravelry.com/stores/boyland-knitworks',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/boylandknitworks/729965112/9A2E3CE2-5BA7-4D29-A339-70015478035B_small2.jpeg']},
 'sheer-v-sweater': {'id': 970742,
  'url': 'https://www.ravelry.com/stores/jessie-mae-martinson-jessie-maed-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jessiemaed/658236447/12B66872-546B-4D2E-B103-C3FE4F96F57A_small2.jpeg']},
 'palm-canyon-video-sweater-class': {'id': 1013214,
  'url': 'https://pattylyons.com/2018/11/meet-the-jetson-sweater-kit/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PattyLyons/686337497/2252_zucker20200229edit_small2.jpg']},
 'stockholm-slipover-v-neck': {'id': 1033141,
  'url': None,
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/703141248/vneck3_small2.jpg']},
 'calliope-13': {'id': 1063027,
  'url': 'https://www.ravelry.com/stores/espace-tricot-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/EspaceTricot/725599973/IMG_8249_small2.jpg']},
 'rituals': {'id': 1074644,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/jojilocat/734283882/WN7A8658_small2.jpg']},
 'eila-3': {'id': 1082708,
  'url': 'https://www.pompommag.com/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Aisleight/740184942/EILA_2x2.GARTER.RIB_IMG_2303_ReadySetRaglan_PomPomPublishing_small2.jpg']},
 'baby-sophisticate-2': {'id': 117515,
  'url': 'http://stockinette.wordpress.com/',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/LindenHeflin/7652545/IMG_0036_small2.JPG']},
 'derecho-2': {'id': 719295,
  'url': 'http://www.berroco.com/patterns/pattern-booklets/386-berroco-remix-light',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/alisonknits/418075273/6061_Remix_Derecho_small2.jpg']},
 'elevenses': {'id': 1006484,
  'url': 'http://www.ravelry.com/stores/frogginette-knitting-patterns',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/frogginette/682916804/ElevensesMain_small2.jpeg']},
 'lunenburg-pullover': {'id': 971392,
  'url': 'http://www.ravelry.com/stores/savory-knitting',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/savoryknitting/654709531/IMG_9621_small2.jpeg']},
 'kukka-cardigan': {'id': 955197,
  'url': 'https://www.ravelry.com/stores/rosa-p',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/rosape/643538634/Kukka1_small2.jpg']},
 'terrazzo-sweater': {'id': 1033464,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/744652962/terrazzo_sweater_5_small2.jpeg']},
 'erell-jumper': {'id': 866637,
  'url': 'https://www.ravelry.com/stores/along-avec-anna---knitting-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/alongavecanna/550477779/IMG_0145_small2.JPG']},
 'arachne-6': {'id': 954781,
  'url': 'https://untangling-knots.com/project/spooky/',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/AndiSatt/643662542/Arachne_Front_V_02_M_small2.jpg']},
 'painting-bricks-sweater': {'id': 1046579,
  'url': 'http://www.ravelry.com/stores/westknits',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/westknits/712272745/PaintingBricksSweater20200422_0462_2048px_small2.jpg']},
 'mandelbrot-2': {'id': 1087895,
  'url': None,
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/Ciospy/743537489/000_small2.jpg']},
 'penguono': {'id': 476481,
  'url': '',
  'photo urls': ['https://images4-g.ravelrycache.com/flickr/1/2/9/12952587564/12952587564_n.jpg']},
 'balloon-cardigan': {'id': 832318,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/PetiteKnitDK/556050919/Balloncardigan_6_small2.jpg']},
 'lillet': {'id': 895432,
  'url': 'http://babycocktails.blogspot.com/',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/babycocktails/598835319/IMG_0599_small2.JPG']},
 'madelyn-cardigan': {'id': 1088023,
  'url': '',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/pattyj/743782118/IMG_20201120_133803638_HDR_small2.jpg']},
 'tilted-tunic': {'id': 1087636,
  'url': 'https://www.ravelry.com/stores/i-need-it-crochet-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/INeedItCrochet/743516758/Photo_1605711039702_small2.jpg']},
 'breathing-space': {'id': 646121,
  'url': 'http://www.ravelry.com/stores/rain-knitwear-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/veera/358302208/Breathing_Space_1_small2.jpg']},
 'vneck-boxy': {'id': 707750,
  'url': 'http://www.ravelry.com/stores/j-o-j-i',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/jojilocat/411351400/bVBoxy_003_small2.jpg']},
 'balloon-sweater-4': {'id': 821793,
  'url': 'https://www.ravelry.com/stores/petiteknit',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/PetiteKnitDK/680729772/hvid6_small2.jpg']},
 'margot-cardigan-2': {'id': 850420,
  'url': 'https://www.ravelry.com/stores/along-avec-anna---knitting-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/alongavecanna/535323316/IMG_7843_small2.jpg']},
 'yanis': {'id': 980654,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/661203840/BE001B20-A6D6-4F1B-90A6-9195383B5AE2_small2.jpeg']},
 'gianluca': {'id': 1014860,
  'url': 'http://www.ravelry.com/stores/grasflecken',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/lilalu/687266027/C274C3CC-E9E2-4CFB-84D5-816E9C0191E1_small2.jpeg']},
 'kaleidoscope-sweater-3': {'id': 1019946,
  'url': 'http://www.ravelry.com/stores/knit-collage',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/knitcollage/692884552/10_Kaleidoscope_Sweater_small2.jpg']},
 'sayuri-sweater-2': {'id': 1063846,
  'url': 'https://www.ravelry.com/stores/valentina-bogdanova-designs',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/valentinasknits/726512453/04_1_small2.jpg']},
 'bromelia-sweater': {'id': 1079754,
  'url': 'http://www.ravelry.com/stores/aksenik-lyudmila-designs',
  'photo urls': ['https://images4-f.ravelrycache.com/uploads/MrsKnowall/738181672/IMG_0526_small2.JPG']},
 'sunnyside': {'id': 241733,
  'url': 'http://www.ravelry.com/stores/tanis-fiber-arts-2',
  'photo urls': ['https://images4-g.ravelrycache.com/uploads/cherylmac/72381820/DSC_0593_small2.JPG']}}

In [5]:
import projects as proj

In [11]:
link = 'https://www.ravelry.com/projects/' + str(proj.proj_by_pattern[id]['projects'][i]['user_id']) + '/' + str(proj.proj_by_pattern[id]['projects'][i]['id'])


In [13]:
len(proj.proj_by_pattern[781496]['projects'])

25

In [29]:
for key in pattern_dict.keys():
    pattern_dict[key]['projects'] = []
    if pattern_dict[key]['id'] in proj.proj_by_pattern.keys():
        pat_id = pattern_dict[key]['id']
        projects_list = proj.proj_by_pattern[pat_id]['projects']
        for i in range(len(projects_list)):
            link = 'https://www.ravelry.com/projects/' \
            + str(projects_list[i]['user_id']) + '/' + str(projects_list[i]['id'])
            pattern_dict[key]['projects'].append(link)


In [30]:
pattern_dict['stripes-32']

{'id': 1087512,
 'url': 'https://www.ravelry.com/stores/dreareneeknits',
 'photo urls': ['https://images4-g.ravelrycache.com/uploads/dreareneeknits/743362891/Custom-8_small2.jpg'],
 'projects': ['https://www.ravelry.com/projects/3879416/26718553',
  'https://www.ravelry.com/projects/4054819/26517636',
  'https://www.ravelry.com/projects/3822345/27047540',
  'https://www.ravelry.com/projects/5746096/26621684',
  'https://www.ravelry.com/projects/2041116/26433234',
  'https://www.ravelry.com/projects/5269278/27053382',
  'https://www.ravelry.com/projects/3905584/26697236',
  'https://www.ravelry.com/projects/196735/26697057',
  'https://www.ravelry.com/projects/407745/27053302',
  'https://www.ravelry.com/projects/334117/27044873',
  'https://www.ravelry.com/projects/3905584/26633703',
  'https://www.ravelry.com/projects/2091/26455553',
  'https://www.ravelry.com/projects/8387885/26506014',
  'https://www.ravelry.com/projects/709020/26426877',
  'https://www.ravelry.com/projects/522568/2

In [ ]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}

for key in pattern_dict.keys():
    if len(pattern_dict[key]['projects']) > 0:
        for i in range(len(pattern_dict[key]['projects'])):
            link = pattern_dict[key]['projects'][i]
            result = requests.get(link, headers = user_agent)
            soup = BeautifulSoup(result.content, 'html.parser')

In [31]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}

link = 'https://www.ravelry.com/projects/280849/26488139'
result = requests.get(link, headers = user_agent)
soup = BeautifulSoup(result.content, 'html.parser')
[image['src'] for image in soup.find('div', class_ = 'photo_gallery_container').find_all('img')]

AttributeError: 'NoneType' object has no attribute 'find_all'

Projects are behind a login - will need to figure out an automated way to query the API for specific projects in order to pull projects in any serious amount.

Classes of sweaters are seriously imbalanced after I've done some binning.
1.  6 librarian cardigan (round neck cardigan with straight vertical opening)
2. 4 v-neck cardigan
3. 91 colorwork round neck sweater
4. 3 boatneck sweater
5. 14 librarian cardigan with fancy stitching
6. 45 round neck sweater with fancy stitching
7. 7 mock neck sweater
8. 22 round neck sweater
9. 1 shawl collar cardigan
10. 7 dolman sleeve sweater
11. 8 shawl collar cardigan with fancy stitching
12. 1 colorwork shawl collar cardigan
13. 2 dolman sleeve sweater with fancy stitching
14. 5 colorwork librarian cardigan
15. 5 unstructured cardigan (straight from sides of neck to hem, no collar)
16. 2 colorwork shawl collar cardigan
17. 6 unstructured cardigan with fancy stitching
18. 1 hooded cardigan
19. 6 colorwork unstructured cardigan
20. 1 colorwork hoodie
21. 1 hoodie
22. 2 v-neck cardigan with fancy stitching
23. 10 v-neck sweater

I'm going to set up lists for each of the 23 classes (permalink), then manually search for and add in other sweaters to fill out the lists - shooting for a minimum of 10 sweaters in each bin, which should add up to at least 30 pictures per bin.

In [49]:
directory = 'img/'
type_dict = {}
for i in range(1, 24):
    type_dict[str(i).zfill(2)] = []
permalink = re.compile(r'(?<=\d\d)[-A-z\d]+(?=_)')
for entry in os.scandir(directory):
    if entry.path.endswith('.jpg') and entry.is_file():
        code = entry.path[4:6]
        type_dict[code] += [str(entry.path)[6:-8]]
type_dict

{'01': ['copenhagen-cardigan',
  'dont-ask',
  'felix-cardigan',
  'a-warm-welcome',
  'kukka-cardigan',
  'carbeth-cardigan'],
 '02': ['harvest-10',
  'april-cardigan-2',
  'aimee-cardigan',
  'balloon-cardigan'],
 '03': ['painting-bricks-sweater',
  'sunbeam-sweater-2',
  'breathing-space',
  'dog-star',
  'mountain-mist-2',
  'suolaulu-sweater',
  'snow-falling-on-cedars',
  'arachne-6',
  'darkwater',
  'zweig',
  'fern--feather',
  'knitorious-rbg',
  'forestland',
  'soundtrack-2',
  'marettimo',
  'missoni-accomplished',
  'aurealis-2',
  'simple-something',
  'panglossian-sweater',
  'le-pouf-sweater',
  'sunset-highway',
  'guell',
  'rug-11',
  'graphic-elements',
  'ingalls',
  'yanis',
  'ghost-horses',
  'hinterland-5',
  'altheda-2',
  'meadow-moon',
  'soldotna-crop',
  'clapton',
  'strange-brew',
  'manou',
  'koivua',
  'sipila',
  'jupiter-crop',
  'goldwing-2',
  'junction-3',
  'mabrey-sweater',
  'alaska-sweater',
  'night-groove',
  'pink-velvet',
  'gardengate',

In [51]:
type_dict['01']

['copenhagen-cardigan',
 'dont-ask',
 'felix-cardigan',
 'a-warm-welcome',
 'kukka-cardigan',
 'carbeth-cardigan']

In [52]:
type_dict['01'] += ['ramona-cardigan', 'dexter-3', 'marum-knitted-bomber', 'kimes', 
                    'quintessential-cardigan-2', 'vanilla-baby', 
                    'plain-and-simple-baby-cardigan', 'order-of-the-garter', 
                   '-9725-neck-down-cardigan-for-women', 'babys-raglan-sweater-no-seams', 
                   'ivy-30', '-221-summer-cardigan', 'kitten-4', 'garter-yoke-baby-cardi']

In [53]:
type_dict['02']

['harvest-10', 'april-cardigan-2', 'aimee-cardigan', 'balloon-cardigan']

In [54]:
type_dict['02'] += ['honey-bop', 'shapely-boyfriend', 'foam-3', 'longfellow-2', 
                   'magnolia-cardigan-3', 'home-base', 'go-to-cardigan', 
                   'fundamental-top-down-mens-cardigan', 'walk-the-line-3', 
                   '16-v-neck-cardigan']

In [56]:
type_dict['04']

['navelli', 'lillet', 'aldous-2']

In [57]:
type_dict['04'] += ['sunshine-coast', 'the-easy-one', 'pebble-tunic-2', 'amory-2', 
                   'seacoast', 'simple-tee-2', 'hayward', 'ridgecrest-pullover', 
                   'davis-2']

In [58]:
type_dict['07']

['terrazzo-sweater',
 'the-classic-2',
 'metropolis-3',
 'novice-sweater',
 'balloon-sweater-4',
 'louisiana-sweater',
 'strathcona-sweater']

In [59]:
type_dict['07'] += ['childhood-2', 'big-kids-bulky-mock-turtleneck', 'opuntia-2', 
                   'raspberry-and-cream', 'gertrude-8', 'nske', 'rainier', 
                   'boundary-pullover', 'refine', 'janet-leigh']

In [60]:
type_dict['09']

['baby-sophisticate-2']

In [61]:
type_dict['09'] += ['georgetown', 'tamarack-for-him', 'soft-flow', 'ponti', 'iced', 
                   'sunday-drive-2', 'sigrid-11', 'mille-for-men', 'shruggigan', 
                   'diary', 'stretta', 'shawl-collar-cardigan-21', 
                   'running-water', 'maestro', 'smokin', 'invert-cardigan']

In [62]:
type_dict['10']

['beyond-the-dunes',
 'boxy',
 'paloma-15',
 'turtle-dove-ii',
 'diaphanous-raglan',
 'cambito',
 'turtle-dove-3']

In [66]:
type_dict['10'] = ['beyond-the-dunes', 'boxy', 'paloma-15', 'turtle-dove-ii', 
                   'diaphanous-raglan', 'cambito', 'turtle-dove-3', 'chaika', 
                   'auer-3', 'freewheeler', 'bibbe', 'soren-4', 
                   'daft-desire', 'bianka', 'mio-5', 'soothe-2', 'vivian-sweater-2', 
                   'flugel', 'mila-7', 'benedicte-soft-dolman-sleeve-sweater']

In [67]:
type_dict['11'] += ['cabin-life-cardigan', 'bees-knees-6', 'jones-3', 'ausable-3', 
                   'noreaster', 'snoqualmie', 'little-wave', 'dark-and-stormy', 
                   'maleme-cardigan', 'glen-shiel-cardigan', 'my-everything', 
                   'migration-7', 'roosevelt-cardigan', 'chocolate-stout', 
                   'radmere', 'augusta-8', 'kathe-cardigan', 'berlin-soft', 
                   'campfire-cardigan-2', 'cold-spring-2']

In [70]:
type_dict['12']

['comfort-fade-cardi']

In [71]:
type_dict['12'] += ['spark-5', 'forest-park-cardigan', 'pascal-5', 
                   'it-takes-a-guild-cardigan-tutorial', 'eaves-2', 
                   'forest-park-junior', 'fireworks-cardi' 'cozy-thoughts-cardi', 
                   'sloane-square-2', 'paintbox-cardigan-2', 'writers-block-2', 
                   'arrowhead-cardigan-2', 'betania-cardigan', 'takoma']

In [72]:
type_dict['13']

['ranunculus-3', 'deschain']

In [73]:
type_dict['13'] += ['lorelei-13', 'nicasio-2', 'sophie-pullover', 'celtic-hill', 
                   'poet', 'olive-leaf-pullover', 'arkenstone-2', 'sailing-sweater', 
                   'almina-sweater', 'kareno---', 'ondawa', 'celosia-pullover', 
                   'victoria-44', 'yokester-2', 'jane-morris', 'spellbound-10', 
                   '197-16-agnes-sweater', 'railfence', 'aida-pullover', 
                   'high-notes', 'corinne-8' '12-pull-jacinthe', 'aarni' 'carra']

In [74]:
type_dict['14']

['mariechen',
 'mandelbrot-2',
 'floozy-2',
 'sommarkoftan-2020--the-summer-cardigan-2020',
 'sorbet-cardigan']

In [75]:
type_dict['14'] += ['the-throwback', 'vega-8', 'budding', 'foxthoughts-cardigan', 
                   'flower-buds-cardigan', 'fjell-cardigan', 'marit-4', 
                   'sjon---the-sea', 'nordbakkan', 'tonnach', 
                   'hillary-fluff-and-the-magic-fungi', 'vesle-vinterfjell-cardigan', 
                   'skumring---nightfall', 'the-sinister-kittigan', 'sol-inca', 
                   'hawthorn-6', 'hela-short-cardigan-with-zip', 
                   'baby-fair-isle-cardigan-3', 'saffran-cardigan', 
                    'a-good-vintage-cardigan', 'little-coffee-bean-bulky', 'nani-4', 
                   'joining-bees-and-things', 'iunn', 'revejakken-1701-05', 
                   'glaswegian-cardigan', 'chestnut-9', 'cloud-nine-cardigan', 
                   'crocheted-sinister-catdigan', 'ella-funt', 
                    'playful-stripes-cardigan', 'dagna-cardigan', 'toorie-2', 
                   'picpus']

In [77]:
type_dict['15']

['no-frills-cardigan',
 'girlfriends-cardigan-anke',
 'newborn-vertebrae',
 'gloam',
 'madelyn-cardigan']

In [78]:
type_dict['15'] += ['girlfriends-cardigan-anke', 'classy-cardi-3', 'growing', 
                   'the-downtown-cardigan', 'fine-sand', 'velvet-cardi', 
                   'summer-longing', 'rhinebeck-cardigan-in-dream', 'whippet-long', 
                   'waits', 'adelheid-cardigan', 'rosemont-cardigan', 
                   'ribtastic-cardi', 'stiina-cardigan', 
                   'breckenridge-topper-cardigan', 'tap-cardigan']

In [81]:
type_dict['12'] += type_dict['16']

In [82]:
del type_dict['16']
type_dict['17']

['reagan',
 'hitofude-cardigan',
 'like-a-cloud',
 'little-love-3',
 'nigella-8',
 'womens-cabled-cardigan']

In [83]:
type_dict['17'] += ['like-a-cloud', 'great-love', 'long-love', 'huia', 
                    'hudson-valley-cardi-video-sweater-class', 'soyokaze-2', 
                    'the-dwell-sweater', 'cliffside-cardigan', 'kapsel', 
                    'gin-and-juice', 'the-scotlander', 'ebbetts', 'greystone-4', 
                    'savage-heart-cardigan', '123-lucky-cardi', 'boho-texture-cardi', 
                    'campsite-waffle-cardigan', '405-lex', 'delaney-cardigan', 
                    'ullock', 'mams-cardigan', 'cafe-au-lait-cardigan', 'zopflicious']


In [84]:
type_dict['18']

['hooded-baby-cardigan-5']

In [85]:
type_dict['18'] += ['artichoke-french-hoodie', 'cropped-hoodie-2', 'tuk', 
                   'everything-nice-hoodie', 'hood-on-the-moon', 'kol', 
                   'dryad-10', 'twilight-new-moon-bella-hooded-cardigan', 
                   'well-water-hoodie', 'kem', 'harumachi', 'billie-5', 
                   'hydes-hooded-sweater', 'sugarcoated', 'gilgalad', 
                   'the-albatross-hooded-cardigan', 'hoodedcardielly', 
                   'jb601-cardigan-with-hood', 'autumn-hooded-cardigan', 
                   'starla-cardi']

In [86]:
type_dict['19']

['rose-74', 'simple-hug-cardi', 'pengweeno', 'le-pouf-2', 'letho', 'penguono']

In [87]:
type_dict['19'] += ['plaid-party-cardi', 'boho-style-mosaic-cardigan', 
                    'zola-jacket', 'eva-cardigan', 'hands', 'big-flower-4', 
                   'nabeela', 'pia-10', 'hearts-balloon-sleeve-cardigan', 
                   'leysingar', 'rattigan', 'sand-and-sunsets', 
                    'desert-nights-cardigan', 'lazulite', 'creekside-3', 
                   'wendela', 'unchained-2', 'hartwick', 'wild-autumn-cardigan', 
                   'bluesand-cardigan', 'flipside-cardigan', 'made-my-day-cardigan', 
                   'emilys-dream', 'windward-5', 'mercantile-cardigan', 
                   '07-gradient-cardigan', '03-open-cardigan']

In [88]:
type_dict['20']

['mini-unbearable-hoodie']

In [89]:
type_dict['20'] += ['jills-sweater-and-hood', 'mens--boys-hoodie-sweater', 
                   '63-16-a---sweater-with-hood', 'teriffic-toddler-hoodie-ii', 
                   'striped-lifestyle-sweater-with-hood', 
                    'kcb001-classic-beginnings-seamless-raglan-pullover', 
                   'kooky-critter-hoodie', 'hooded-striped-top', 
                   'winter-hoodie', 'magnus-4', 'cozy-campfire-striped-hoodie', 
                   'kangaroo-hoodie-4', 'marv', 'the-mini-montrealer', 
                   'the-montrealer', 'snowy-owl-hoodie', 'a-wolfs-moon', 
                   't-rex-hoodie', '-41-luisa', 'mouse-hoodie', 'whale-hoodie', 
                   '13---pull-a-capuche', 'hoodie-smile', '172---pull-a-capuche', 
                   'pullover-with-star', 'blue-child-s-sweater-with-roses', 
                   '83-paakallohuppari', 'alston-flower-show-hoodie', 
                   'triceratops-dinosaur-hoodie', 'oh-my-bear', 
                   'velociraptor-dinosaur-hoodie', 'snowy-owl-hoodie', 
                   'rock-star-guitar-hoodie']

In [90]:
type_dict['21']

['november20']

In [91]:
type_dict['21'] += ['-211-baby-tunic', 'a-wonderful-wallaby-for-grown-ups---2016', 
                   'tokyo-hoodie', 'channelside-hoodie-3', 'happy-at-home-hoodie', 
                   'clubhouse-raglan-hoodie', 'milsom', 'windbreak', 'atherton-2', 
                   'east-neuk-hoodie', 'p225-babys-back-zip-hooded-sweater', 
                   'basic-toddler-hoodie', '109-1-campfire', 
                   'garter-stitch-hooded-sweater-shirt', 'pebble-beach', 
                   '32-hooded-pullover', 'sweater-cardigan-and-waistcoat-vest-3996', 
                   'hooded-sweater', 'brilliant-bubble', '05-pullover-hoodie', 
                   'hayley-hoodie', 'pull-a-capuche-femme-438-t5-101', 
                   'hooded-sweatshirt-2', '109-9-river-maid', 'hooded-sweater-21']

In [92]:
type_dict['22']

['rosa-cardigan-5', 'rosa-little']

In [93]:
type_dict['22'] += ['must-have-cardigan', 'jack-rose', 'willa-12', 'anwyn-2', 
                   'vianne-2', '17-topiary', 'gocce-2', 'farrah-fowler', 
                   'spring-quartet', 'geiger', 'dolin', 'gaspra', 'decq', 'irpa', 
                   'boston-cardigan-2', '283-maelynn-cardigan', 'baby-groots-cardigan', 
                   'glenfiddich', 'candelaria', 'boston-cardi-kids', 'marion-6', 
                   'lisboa-soft', 'rhythm-4', '09-chevron-cardigan', 'clematite-2', 
                   'fidelia-4', 'bradshaw-3', 'celyn', 'mary-mead', 
                   'birch-cardigan-2', 'augusta-cardigan', 'alicia-19', 
                   'homelike-cardigan', 'stickad-herrkofta', 'dock-and-cabin', 
                   'woodburne-cardigan', 'shakespeare-cardigan', 'blumchen', 
                   'vignette', 'leaf-and-picot-cardigan', 'dads-cardigan', 
                   'summers-end', 'footlights-cardigan', 'pto-01801-lang-yarns-kylie']

In [94]:
type_dict['23']

['understated-2',
 'niamh-11',
 'sheer-v-sweater',
 'very-v-neck-raglan',
 'cumulus-blouse',
 'stockholm-slipover-v-neck',
 'ursa-3',
 'ursina',
 'the-v-back-tee',
 'vneck-boxy']

In [95]:
type_dict['23'] += ['hvide', 'edgartown-light', 'weir-2', 'anyday-sweater', 
                   'dillon-2', 'westbound', 'ursa-minor-4', 'rock-it-tee-dk', 
                   'simple-summer-tweed-top-down-v-neck', 'bairbre-2']

In [97]:
ua = UserAgent()
import random

for key in type_dict.keys():
    for pattern in type_dict[key]:
        link = 'https://www.ravelry.com/patterns/library/' + pattern
        user_agent = {'User-agent': ua.random}
        result = requests.get(link, user_agent)
        soup = BeautifulSoup(result.content, 'html.parser')
        images = [image['src'] for image in soup.find('div', class_ = 'photo_gallery_container').find_all('img')]
        count = 2
        for image in images:
            img_name = 'img/' + key + pattern + '_' + str(count).zfill(3) + '.jpg'
            try:
                urllib.request.urlretrieve(image, img_name)
            except:
                pass
            count += 1
            time.sleep(1.5*random.random())
        time.sleep(5*random.random())
            
            
            

AttributeError: 'NoneType' object has no attribute 'find_all'

In [102]:
type_dict['12'].sort()
type_dict['12']

['arrowhead-cardigan-2',
 'betania-cardigan',
 'comfort-fade-cardi',
 'eaves-2',
 'fireworks-cardicozy-thoughts-cardi',
 'forest-park-cardigan',
 'forest-park-junior',
 'gramps',
 'it-takes-a-guild-cardigan-tutorial',
 'paintbox-cardigan-2',
 'pascal-5',
 'sloane-square-2',
 'spark-5',
 'spark-5',
 'takoma',
 'writers-block-2']

In [104]:
type_dict2 = {}
type_dict2['12'] = ['arrowhead-cardigan-2', 'betania-cardigan', 
                    'fireworks-cardicozy-thoughts-cardi', 'gramps', 
                   'paintbox-cardigan-2', 'sloane-square-2', 
                   'takoma', 'writers-block-2']
type_dict2['13'] = type_dict['13']
type_dict2['14'] = type_dict['14']
type_dict2['15'] = type_dict['15']
type_dict2['17'] = type_dict['17']
type_dict2['18'] = type_dict['18']
type_dict2['19'] = type_dict['19']
type_dict2['20'] = type_dict['20']
type_dict2['21'] = type_dict['21']
type_dict2['22'] = type_dict['22']
type_dict2['23'] = type_dict['23']

In [105]:
for key in type_dict2.keys():
    for pattern in type_dict2[key]:
        link = 'https://www.ravelry.com/patterns/library/' + pattern
        user_agent = {'User-agent': ua.random}
        result = requests.get(link, user_agent)
        try:
            soup = BeautifulSoup(result.content, 'html.parser')
            images = [image['src'] for image in soup.find('div', class_ = 'photo_gallery_container').find_all('img')]
            count = 2
            for image in images:
                img_name = 'img/' + key + pattern + '_' + str(count).zfill(3) + '.jpg'
                try:
                    urllib.request.urlretrieve(image, img_name)
                except:
                    pass
                count += 1
                time.sleep(1.5*random.random())
        except:
            pass
        time.sleep(5*random.random())


ConnectionError: HTTPSConnectionPool(host='www.ravelry.com', port=443): Max retries exceeded with url: /patterns/library/poet?User-agent=Opera%2F9.80+%28Macintosh%3B+Intel+Mac+OS+X+10.6.8%3B+U%3B+de%29+Presto%2F2.9.168+Version%2F11.52 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb9fa792c10>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [107]:
sorted(type_dict2['13'])

['197-16-agnes-sweater',
 'aarnicarra',
 'aida-pullover',
 'almina-sweater',
 'arkenstone-2',
 'celosia-pullover',
 'celtic-hill',
 'corinne-812-pull-jacinthe',
 'deschain',
 'high-notes',
 'jane-morris',
 'kareno---',
 'lorelei-13',
 'nicasio-2',
 'olive-leaf-pullover',
 'ondawa',
 'poet',
 'railfence',
 'ranunculus-3',
 'sailing-sweater',
 'sophie-pullover',
 'spellbound-10',
 'victoria-44',
 'yokester-2']

In [108]:
type_dict2['13'] = ['197-16-agnes-sweater', 'aarnicarra', 'aida-pullover', 
                    'almina-sweater', 'arkenstone-2', 'celosia-pullover', 
                    'celtic-hill', 'corinne-812-pull-jacinthe', 'high-notes', 
                    'jane-morris', 'kareno---', 'lorelei-13', 
                    'olive-leaf-pullover', 'ondawa', 'poet', 'railfence', 
                    'sailing-sweater', 'sophie-pullover', 'spellbound-10', 
                    'victoria-44', 'yokester-2']

In [110]:
del type_dict2['12']
skipped = {'13': [], '14': [], '15': [], '17': [], '18': [], 
           '19': [], '20': [], '21': [], '22': [], '23': []}

for key in type_dict2.keys():
    for pattern in type_dict2[key]:
        link = 'https://www.ravelry.com/patterns/library/' + pattern
        user_agent = {'User-agent': ua.random}
        try:
            result = requests.get(link, user_agent)
            soup = BeautifulSoup(result.content, 'html.parser')
            images = [image['src'] for image in soup.find('div', class_ = 'photo_gallery_container').find_all('img')]
            count = 2
            for image in images:
                img_name = 'img/' + key + pattern + '_' + str(count).zfill(3) + '.jpg'
                try:
                    urllib.request.urlretrieve(image, img_name)
                except:
                    pass
                count += 1
                time.sleep(1 + 2*random.random())
        except:
            skipped[key] += [pattern]
        time.sleep(2 + 5*random.random())
